In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('ip.xlsx')
df.drop(["Section Spec path","File Name"], axis=1, inplace=True)
df.columns
df.dtypes

Job's Name                       object
System                           object
Manufacturer                     object
Section Spec                     object
SOW Keywords                     object
Primary product keywords         object
Primary product PDS URL          object
Accessories products keywords    object
Accessories product PDS URL      object
Section Spec number               int64
Content                          object
words                             int64
dtype: object

In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
        FieldSchema(name='id', dtype=DataType.INT64, description='id', is_primary=True, auto_id=False, dim=dim),
        FieldSchema(name='J_name', dtype=DataType.VARCHAR, description='Job\'s Name', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='System', dtype=DataType.VARCHAR, description='System', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='Manufacturer', dtype=DataType.VARCHAR, description='Manufacturer', is_primary=False, auto_id=False, max_length=500, dim=dim),
        FieldSchema(name='Section_Spec', dtype=DataType.VARCHAR, description='Section Spec', is_primary=False, auto_id=False, max_length=500, dim=dim),
        FieldSchema(name='SOW_key', dtype=DataType.VARCHAR, description='SOW Keywords', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='product_key', dtype=DataType.VARCHAR, description='Primary product keywords', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='product_URL', dtype=DataType.VARCHAR, description='Primary product PDS URL', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='acc_key', dtype=DataType.VARCHAR, description='Accessories products keywords', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='acc_URL', dtype=DataType.VARCHAR, description='Accessories product PDS URL', is_primary=False, auto_id=False, max_length=500 , dim=dim),
        FieldSchema(name='Section_Spec_no', dtype=DataType.INT64, description='Section Spec number', is_primary=False, auto_id=False , dim=dim),
        FieldSchema(name='Content', dtype=DataType.FLOAT_VECTOR, description='Content', is_primary=False, auto_id=False, max_length=10000 , dim=dim),
        FieldSchema(name='words', dtype=DataType.INT64, description='words', is_primary=False, auto_id=False , dim=dim),
    ]
    schema = CollectionSchema(fields=fields, description='Roofing Section Spec content Data')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'IP',
        'index_type':"GPU_IVF_FLAT",
        'params':{"nlist":130}
    }
    collection.create_index(field_name='Content', index_params=index_params)
    return collection

collection = create_milvus_collection('Roofing_section_data', 768)

print("Collection {} created".format(collection.name))

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19530, illegal connection params or server unavailable)>

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
        FieldSchema(name='id', dtype=DataType.INT64, description='id', is_primary=True, auto_id=False, dim=dim),
        FieldSchema(name='Content', dtype=DataType.FLOAT_VECTOR, description='Content', is_primary=False, auto_id=False, max_length=10000 , dim=dim),
    ]
    schema = CollectionSchema(fields=fields, description='ONly content data with id ')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'IP',
        'index_type':"GPU_IVF_FLAT",
        'params':{"nlist":130}
    }
    collection.create_index(field_name='Content', index_params=index_params)
    return collection

collection = create_milvus_collection('Content_data', 768)

print("Collection {} created".format(collection.name))

Collection Content_data created


 """   from towhee import ops, pipe, DataCollection , AutoPipes

    insert_pipe = (
        pipe.input('Content')
            .map('Content','Content_vec',ops.sentence_embedding.transformers(model_name='bert-base-uncased'))
            .output('Content_vec')
    ) """

from towhee import ops, pipe, DataCollection , AutoPipes

insert_pipe_milvus = (
    pipe.input('df')
        .flat_map('df' , 'data', lambda df: df.values.tolist())
        .map('data','embedding',ops.sentence_embedding.transformers(model_name='bert-base-uncased'))
        .output('embedding')
)

In [2]:
# add id column to the dataframe
df['id'] = df.index
df = df[['id','Content']]

data = df.to_dict('records')

NameError: name 'df' is not defined

In [ ]:
from towhee import ops, pipe, DataCollection , AutoPipes
insert_pipe = (
    pipe.input('id', 'Content')
        .map('Content', 'vec', ops.sentence_embedding.transformers(model_name='bert-base-uncased'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec'), 'insert_status', ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='Content_data'))
        .output('insert_status')
)

In [ ]:
ids = df['id'].tolist()
Content = df['Content'].tolist()
for i in range(len(Content)):
    result = insert_pipe(ids[i], Content[i])

In [ ]:
result

In [ ]:
# create a fuction use content column to create bert embedding and insert into milvus

from sentence_transformers import SentenceTransformer
import pandas as pd

def create_bert_embeddings(df, column):
    # Initialize the BERT model
    model = SentenceTransformer('bert-base-uncased')

    # Create embeddings for the specified column
    embeddings = model.encode(df[column].tolist())

    # Create a new DataFrame with the embeddings
    df_embeddings = pd.DataFrame(embeddings.tolist(), columns=[f'feature_{i}' for i in range(embeddings.shape[1])])

    return df_embeddings

In [ ]:
df_embeddings = create_bert_embeddings(df, 'Content')

2024-04-22 12:14:30,192 - 139051837343552 - SentenceTransformer.py-SentenceTransformer:1192 - WARNING: No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.


In [ ]:
ids = df['id'].tolist()
embeddings = df_embeddings.values.tolist()
mr = collection.insert(data=[ids, embeddings] , timeout=10)
# Flush the collection to make sure data is persisted to disk
collection.load()

In [ ]:
mr.insert_count

# flush 
collection.flush()

In [ ]:
collection.load()
data= collection.is_empty


In [ ]:
data

False

In [ ]:
collection.num_entities

52

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


# Function to search for similarity in the vector database
def search_similarity(input_text, df_embeddings):
    model = SentenceTransformer('bert-base-uncased')
    input_embedding = model.encode([input_text])
    
    # Calculate cosine similarity between input embedding and database embeddings
    similarities = cosine_similarity(input_embedding, df_embeddings.values)
    
    # Get indices of most similar entries
    most_similar_indices = similarities.argsort()[0][::-1]
    
    # Get the most similar entries from the vector database
    most_similar_entries = df_embeddings.iloc[most_similar_indices]
    
    return most_similar_entries

# Example usage
input_text = '''your input text here'''
most_similar_results = search_similarity(input_text, df_embeddings)
print(most_similar_results)

/home/ubuntu/anaconda3/envs/RPM-POC/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'df_embeddings' is not defined

In [4]:
# create dict from df to insert into milvus
for row in df.iterrows():
    data = row[1].to_dict()
    print(data)

data = [
    {
        
    }
]

NameError: name 'df1' is not defined

In [5]:
df1

NameError: name 'df1' is not defined